In [66]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

In [67]:
# Sample Inputs
total_investment = 100000  # Total investment amount
start_date = "2024-05-01"  # Investment period start date
end_date = "2024-05-30"  # Investment period end date
end_date2 = "2024-05-28"  # Investment period end date

In [68]:
stocks = [
	{"Ticker": "ADANIPORTS.NS", "Weightage": 35.0000},
	{"Ticker": "APOLLOHOSP.NS", "Weightage": 30.0000},
	{"Ticker": "ASIANPAINT.NS", "Weightage": 35.0000},]


In [69]:
def calculate_rsi(data, periods=14):
	"""
	Calculates the Relative Strength Index (RSI) for a given DataFrame.

	Args:
		data: pandas DataFrame containing the 'Close' price data.
		periods: Number of periods for RSI calculation (default: 14).

	Returns:
		pandas Series containing the RSI values.
	"""

	delta = data["Close"].diff()
	gain = (delta.where(delta > 0, 0)).fillna(0)
	loss = (delta.where(delta < 0, 0)).fillna(0)

	avg_gain = gain.rolling(window=periods).mean()
	avg_loss = abs(loss.rolling(window=periods).mean())

	rs = avg_gain / avg_loss
	rsi = 100 - (100 / (1 + rs))

	print(rsi)

	return rsi

In [70]:
def allocate_investment(adjust_weightage_by_rsi, total_investment):
    """
    Allocates investment amount based on adjusted weights.

    Args:
        adjust_weightage_by_rsi: A list of adjusted weights for each stock.
        total_investment: The total amount to be invested.

    Returns:
        A list of allocated amounts for each stock.
    """
    total_weight = sum(adjust_weightage_by_rsi)
    allocated_amounts = [
        weight / total_weight * total_investment for weight in adjust_weightage_by_rsi
    ]
    return allocated_amounts


# Adjusted start date for RSI calculation
extended_start_date = pd.to_datetime(start_date) - pd.Timedelta(days=14)
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Daily investment (total investment divided by the number of days)
total_days = (end_date - start_date).days + 1
daily_investment = total_investment / total_days


# Function to adjust weightage by RSI
def adjust_weightage_by_rsi(weightage, rsi):
    if rsi is None or pd.isna(rsi):
        return weightage  # If RSI is missing, return the original weightage

    rsi_ranges = {
        (90, 100): 0.7,  # Reduce by 30%
        (80, 90): 0.8,  # Reduce by 20%
        (70, 80): 0.9,  # Reduce by 10%
        (0, 10): 1.3,  # Increase by 30%
        (10, 20): 1.2,  # Increase by 20%
        (20, 30): 1.1,  # Increase by 10%
    }

    for (low, high), adjustment in rsi_ranges.items():
        if low <= rsi < high:
            return weightage * adjustment

    return weightage  # No change if between 30 and 70


# Results list for investment details
results = []

# Loop through each stock
for stock in stocks:
    ticker = stock["Ticker"]
    weightage = stock["Weightage"]

    # Download stock data
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        print(f"Data for {ticker} is not available. Skipping.")
        continue

    # Calculate RSI
    data["RSI"] = calculate_rsi(data, periods=14)

    # Adjust weightage dynamically
    data["Adjusted Weightage"] = data["RSI"].apply(
        lambda x: adjust_weightage_by_rsi(weightage, x)
    )

    # Daily allocated investment (based on daily_investment)
    data["Allocated Investment"] = data["Adjusted Weightage"] * daily_investment

    # Safely calculate shares (consider handling zero close prices)
    data["Shares"] = data.apply(
        lambda row: (
            row["Allocated Investment"] // row["Close"].iloc[0]
            if row["Close"].iloc[0] > 0
            else 0
        ),
        axis=1,
    )

    # Append results with additional investment details
    for index, row in data.iterrows():
        results.append(
            {
                "Ticker": ticker,
                "Date": index.date(),
                "Weightage": weightage,
                "Adjusted Weightage": row["Adjusted Weightage"].iloc[0],
                # Uncomment if normalized weightage is implemented
                # "Normalized Weightage": row.get("Normalized Weightage", None),
                "Allocated Investment": row["Allocated Investment"],
                "Open Price": row["Open"].iloc[0],
                "Close Price": row["Close"].iloc[0],
                "RSI": row["RSI"].iloc[0],
                "Shares": row["Shares"].iloc[0],
            }
        )

# Convert results to DataFrame
summary = pd.DataFrame(results)

# Reorder and display columns dynamically
columns = [
    "Ticker",
    "Date",
    "Weightage",
    "Adjusted Weightage",
    "Allocated Investment",
    "Open Price",
    "Close Price",
    "RSI",
    "Shares",
]
if "Normalized Weightage" in summary.columns:
    columns.insert(4, "Normalized Weightage")
summary = summary[columns]

# Save the summary to CSV
summary.to_csv("investment_summary.csv", index=False)

print("\nSummary saved to 'investment_summary.csv'")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Ticker      ADANIPORTS.NS
Date                     
2024-05-02            NaN
2024-05-03            NaN
2024-05-06            NaN
2024-05-07            NaN
2024-05-08            NaN
2024-05-09            NaN
2024-05-10            NaN
2024-05-13            NaN
2024-05-14            NaN
2024-05-15            NaN
2024-05-16            NaN
2024-05-17            NaN
2024-05-21            NaN
2024-05-22      57.542089
2024-05-23      66.018057
2024-05-24      64.294220
2024-05-27      72.119887
2024-05-28      66.160660
2024-05-29      69.059570
Ticker      APOLLOHOSP.NS
Date                     
2024-05-02            NaN
2024-05-03            NaN
2024-05-06            NaN
2024-05-07            NaN
2024-05-08            NaN
2024-05-09            NaN
2024-05-10            NaN
2024-05-13            NaN
2024-05-14            NaN
2024-05-15            NaN
2024-05-16            NaN
2024-05-17            NaN
2024-05-21            NaN
2024-05-22      42.854904
2024-05-23      50.782844
2024-05-24  